In [27]:
import nltk
import string
import pandas as pd
import numpy as np
import itertools
from nltk.corpus import words
import pandas as pd
import numpy as np
import string
import fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import ast
import tqdm
from tqdm import tqdm
#nltk.download('words')

In [28]:
nmfp=pd.read_csv('nmfp.csv')
nic=pd.read_csv('nic.csv')
markit=pd.read_csv('markit.csv')
clean_name_fixes=pd.read_csv('clean_name_fixes.csv')
y9c_matched=pd.read_csv('matched_y9c.csv')
y9c_unmatched=pd.read_csv('unmatched_y9c.csv')

In [29]:
#permanent unmathched is the list of clean names that have manually been found to not have a match
#these will be excluded from the automated matching
permanent_unmatched=clean_name_fixes[pd.isnull(clean_name_fixes['values'])]

In [30]:
#clean_name_fixes_dict maps values in the clean_name column of nmfp to their proper match in nic or y9c
clean_name_fixes=clean_name_fixes[pd.notnull(clean_name_fixes['values'])]
clean_name_fixes_dict=dict(zip(clean_name_fixes['keys'],clean_name_fixes['values']))

In [31]:
#remove punctuation, capitals and extra spaces
def first_clean(s):
    translator=str.maketrans('', '', string.punctuation)
    new=str(s).translate(translator).lower().replace('  ',' ')
    return new

In [32]:
#remove words from a string if they are in the list 'words'
def remove_words(s,words):
    temp=s.split()
    new=''
    for i in temp:
        if not i in words:
            new=new+' '+i
    new=new[1:len(new)]
    return new

In [33]:
#words_list has words like ltd, plc, corp, etc. that are common across many entities
words_list=pd.read_csv('words_list.csv',header=None)
#words_list2 has the words in words_list as well as place names to limit false matches when comparing "full_stripped_name"
#which removes all english words
words_list2=pd.read_csv('words_list2.csv',header=None)

In [34]:
#words.words() is a list of all english words from nltk
all_words=[]
for i in words.words():
    all_words.append(i)
    all_words.append(i+'s')

In [35]:
#create the clean_name column in nmfp which will be used for generating a smaller word list
nmfp['clean_name']=pd.Series(first_clean(i) for i in nmfp['nmfp_name'])

In [36]:
#words_list3 has all english words in nmfp as well as common company suffixes and place names
words_list3=list(i.split() for i in nmfp['clean_name'].values)
words_list3=list(itertools.chain(*words_list3))
words_list3=list(set(words_list3).intersection(set(all_words)))
for i in words_list2[0]:
    words_list3.append(i)

In [37]:
#create various kinds of cleaned name columns
nmfp['trimmed_name']=pd.Series(remove_words(i,words_list) for i in nmfp['clean_name'])
nmfp['full_stripped_name']=pd.Series(remove_words(i,words_list3) for i in nmfp['clean_name'])
nic['clean_name']=pd.Series(first_clean(i) for i in nic['short_name'])
nic['clean_long_name']=pd.Series(first_clean(i) for i in nic['name'])
nic['trimmed_name']=pd.Series(remove_words(i,words_list) for i in nic['clean_name'])
nic['full_stripped_name']=pd.Series(remove_words(i,words_list3) for i in nic['clean_name'])
nic['trimmed_long_name']=pd.Series(remove_words(i,words_list) for i in nic['clean_long_name'])
nic['stripped_long_name']=pd.Series(remove_words(i,words_list3) for i in nic['clean_long_name'])
markit['clean_name']=pd.Series(first_clean(i) for i in markit['short_name'])
markit['clean_long_name']=pd.Series(first_clean(i) for i in markit['name'])
markit['trimmed_name']=pd.Series(remove_words(i,words_list) for i in markit['clean_name'])
markit['full_stripped_name']=pd.Series(remove_words(i,words_list3) for i in markit['clean_name'])
nic['clean_long_name']=nic['clean_long_name'].apply(lambda x: x.strip())
nic['clean_name']=nic['clean_name'].apply(lambda x: x.strip())
nmfp['clean_name']=nmfp['clean_name'].apply(lambda x: x.strip())

In [38]:
nmfp

,percent,cumulative_percent,cusip6,isin,lei,cik,rssd_id,ticker,nmfp_name,frb_name,...,markit_fisd_name,nmfp_trace_name,frb_trace_name,nic_trace_name,markit_trace_name,ccm_name,edgar_name,clean_name,trimmed_name,full_stripped_name
0,6.968568,6.968568,912828,NaN,NaN,NaN,NaN,USGB,U.S. Treasury Note,NaN,...,UNITED STATES TREAS SECURITY,NaN,NaN,NaN,NaN,NaN,NaN,us treasury note,us treasury note,
1,6.769854,13.738422,912796,NaN,NaN,NaN,NaN,USGB,UNITED STATES TREASURY,NaN,...,UNITED STATES TREAS SECURITY,NaN,NaN,NaN,NaN,NaN,NaN,united states treasury,united states treasury,
2,2.963436,16.701858,912795,NaN,NaN,NaN,NaN,USGB,UNITED STATES TREASURY,NaN,...,UNITED STATES TREAS SECURITY,NaN,NaN,NaN,NaN,NaN,NaN,united states treasury,united states treasury,
3,2.510771,19.212629,NaN,NaN,NaN,1457970.0,NaN,NaN,Federal Reserve Bank of New York,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,federal reserve bank of new york,federal reserve bank of new york,
4,2.436572,21.649202,313385,NaN,NaN,NaN,NaN,NaN,Federal Home Loan Bank,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,federal home loan bank,federal home loan bank,
5,2.360304,24.009506,313384,NaN,NaN,NaN,NaN,FEDHOM,Federal Home Loan Bank,NaN,...,FEDERAL HOME LN BK CHICAGO,FEDERAL HOME LN BK CONS DISC NTS,NaN,NaN,FEDERAL HOME LN BK CHICAGO,NaN,NaN,federal home loan bank,federal home loan bank,
6,1.771453,25.780959,NaN,NaN,NaN,915092.0,NaN,NaN,Master Investment Portfolio,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,master investment portfolio,master investment portfolio,
7,0.889931,26.670890,NaN,NaN,NaN,1094885.0,NaN,NaN,State Street Money Market Portfolio,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,state street money market portfolio,state street money market portfolio,
8,0.781192,27.452082,912796,NaN,254900HROIFWPRGM1V77,NaN,NaN,USGB,UNITED STATES TREASURY,NaN,...,UNITED STATES TREAS SECURITY,NaN,NaN,NaN,NaN,NaN,NaN,united states treasury,united states treasury,
9,0.763459,28.215541,NaN,NaN,NaN,1403166.0,NaN,NaN,Treasury Master Fund,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,treasury master fund,treasury master fund,


In [12]:
#there is one blank clean_name entry
y9c_matched=y9c_matched[pd.notnull(y9c_matched['clean_name'])]

In [13]:
#remove extra spaces from clean names
y9c_matched['clean_name']=y9c_matched['clean_name'].apply(lambda x: x.strip())
y9c_unmatched['clean_name']=y9c_unmatched['clean_name'].apply(lambda x: x.strip())

In [14]:
#unmatched_rssd is anything left unmatched after the match with y9c
#add full_stripped_name to matched and unmatched because not used in the y9c match
unmatched_rssd=y9c_unmatched.copy()
matched_rssd=y9c_matched.copy()
matched_rssd=matched_rssd[pd.notnull(matched_rssd['clean_name'])]
unmatched_rssd['full_stripped_name']=pd.Series(remove_words(i,words_list3) for i in unmatched_rssd['clean_name'])
matched_rssd['full_stripped_name']=pd.Series(remove_words(i,words_list3) for i in matched_rssd['clean_name'])

In [15]:
#reorganize columns
col_list = list(matched_rssd)
col_list[len(col_list)-1], col_list[len(col_list)-2] = col_list[len(col_list)-2], col_list[len(col_list)-1]
matched_rssd=pd.DataFrame(matched_rssd[col_list])

In [16]:
#remove anything in permanent_unmatched from unmatched
unmatched_rssd.drop(unmatched_rssd[unmatched_rssd['clean_name'].isin(permanent_unmatched['keys'])].index,inplace=True)

In [17]:
unmatched_rssd.reset_index(inplace=True)

In [18]:
matched_rssd.reset_index(inplace=True)

In [19]:
matched_rssd.drop(['Unnamed: 0','index'],axis=1,inplace=True)
unmatched_rssd.drop(['Unnamed: 0','index'],axis=1,inplace=True)

In [20]:
#matched is the df of already matched data
#unmatched is df of unmatched data
#dataset is either nic or markit
#field1 is column from nmfp used for the match
#field2 is column from either nic or markit used for the match
def exact_match(matched,unmatched,dataset,field1,field2):
    temp=unmatched[unmatched[field1].isin(dataset[field2])].copy()
    temp=temp[temp[field1]!='']
    temp=temp[pd.notnull(temp[field1])]
    unmatched.drop(unmatched[unmatched.index.isin(temp.index)].index,inplace=True)
    temp.reset_index(inplace=True)
    temp['matched_name']=pd.Series(list(dataset['clean_long_name'][nic[field2]==i]) for i in temp[field1].values)
    #temp=temp[temp['matched_name']!='']
    temp=temp[pd.notnull(temp['matched_name'])]
    temp=temp.drop('index',axis=1)
    matched=pd.concat([matched,temp],axis=0)
    return matched,unmatched

In [21]:
#match between clean_name columns in nic and unmatched
output=exact_match(matched_rssd,unmatched_rssd,nic,'clean_name','clean_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [22]:
output=exact_match(matched_rssd,unmatched_rssd,nic,'clean_name','clean_long_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [23]:
output=exact_match(matched_rssd,unmatched_rssd,nic,'trimmed_name','trimmed_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [24]:
output=exact_match(matched_rssd,unmatched_rssd,nic,'trimmed_name','trimmed_long_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [25]:
output=exact_match(matched_rssd,unmatched_rssd,nic,'full_stripped_name','full_stripped_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [26]:
output=exact_match(matched_rssd,unmatched_rssd,nic,'full_stripped_name','stripped_long_name')
matched_rssd=output[0]
unmatched_rssd=output[1]

In [27]:
matched_rssd.reset_index(inplace=True)

In [28]:
#convert entries in matched_name column to list so that they can be used for fuzzy matching
matched_rssd['matched_name']=pd.Series([i] if type(i)!=list else i for i in matched_rssd['matched_name'])

In [29]:
#create good_fuzzys column which fuzzy matches between clean_name column (string) and matched_name column (list)
#this is to eliminate false matches obtained from the stripped_name matching
matched_rssd.loc[:,'good_fuzzys']=pd.Series(list(process.extractWithoutOrder(matched_rssd['clean_name'][i],matched_rssd['matched_name'][i],score_cutoff=90)) for i in matched_rssd.index)

In [30]:
#find cases where there were no good fuzzy matches
temp=matched_rssd[matched_rssd['good_fuzzys'].apply(lambda x: len(x)==0)].copy()

In [31]:
#remove anything in temp from matched
matched_rssd.drop(matched_rssd[matched_rssd.index.isin(temp.index)].index,inplace=True)

In [32]:
temp.drop(['index','matched_name','good_fuzzys'],axis=1,inplace=True)

In [33]:
#put all the entries with no fuzzy score>90 back into unmatched
unmatched_rssd=pd.concat([unmatched_rssd,temp],axis=0)

In [34]:
#df1 is the data you want matches for (needle)
#df2 is the data you are looking in (haystack)
#field1 and field2 are the columns in df1 and df2 respectively
def fuzzy_match(df1,df2,field1,field2,fuzzys_dict=''):
    if fuzzys_dict=='':
        letters=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','0','1','2','3','4','5','6','7','8','9']
        letters_list=[]
        for i in range(len(letters)):
            letters_list.append(list(df2[field2][list(j[0]==letters[i] for j in df2[field2])]))
        letters_dict={'a':0,'b':1,'c':2,'d':3,'e':4,'f':5,'g':6,'h':7,'i':8,'j':9,'k':10,'l':11,'m':12,'n':13,'o':14,'p':15,'q':16,'r':17,'s':18,'t':19,'u':20,'v':21,'w':22,'x':23,'y':24,'z':25,'0':26,'1':27,'2':28,'3':29,'4':30,'5':31,'6':32,'7':33,'8':34,'9':35}
        unique=list(set(df1[field1]))
        unique_fuzzys=[]
        for i in unique:
            temp=i[0]
            ind=letters_dict[temp]
            unique_fuzzys.append(list(process.extractWithoutOrder(i,letters_list[ind],score_cutoff=90)))
            if len(unique_fuzzys)%100==0:
                print(len(unique_fuzzys))
        fuzzys_dict=dict(zip(unique,unique_fuzzys))
    all_fuzzys=[]
    for i in df1[field1]:
        if i in fuzzys_dict.keys():
            all_fuzzys.append(fuzzys_dict[i])
        else:
            all_fuzzys.append([])
    all_fuzzys=pd.Series(all_fuzzys)
    all_fuzzys_bool=[]
    for i in all_fuzzys:
        if i==[]:
            all_fuzzys_bool.append(False)
        else:
            all_fuzzys_bool.append(True)
    df1['old_index']=df1.index
    df1.reset_index(inplace=True)
    df1['fuzzys']=all_fuzzys
    temp=df1[all_fuzzys_bool].copy()
    df1.drop('fuzzys',axis=1,inplace=True)
    fuzzy_matched_names=[]
    for i in temp['fuzzys']:
        temp_names=[]
        for j in i:
            temp_names.append(j[0])
        fuzzy_matched_names.append(temp_names)
    temp['matched_name']=fuzzy_matched_names
    temp.drop('fuzzys',axis=1,inplace=True)
    temp.drop('index',axis=1,inplace=True)
    temp.drop('old_index',axis=1,inplace=True)
    temp['good_fuzzys']=temp['matched_name'].copy()
    reverse_bool=[not c for c in all_fuzzys_bool]
    unmatched=df1[reverse_bool].copy()
    return temp,unmatched,fuzzys_dict

In [35]:
#this is the dict generated from the fuzzy match in a previous iteration (much faster to reuse it than re-run)
nmfp_nic_fuzzys=pd.read_csv('nmfp_nic_fuzzys.csv')

In [36]:
nmfp_nic_fuzzys['values']=nmfp_nic_fuzzys['values'].apply(lambda x: ast.literal_eval(x))

In [37]:
nmfp_nic_dict=dict(zip(nmfp_nic_fuzzys['keys'],nmfp_nic_fuzzys['values']))

In [38]:
if 'level_0' in unmatched_rssd.columns:
    unmatched_rssd.drop('level_0',axis=1,inplace=True)
if 'index' in unmatched_rssd.columns:
    unmatched_rssd.drop('index',axis=1,inplace=True)

In [39]:
#fuzzy match between matched and unmatched
output=fuzzy_match(unmatched_rssd,matched_rssd,'clean_name','clean_name')

100
200
300
400
500
600
700


C:\Users\jmh2321\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: generator 'extractWithoutOrder' raised StopIteration
  app.launch_new_instance()


800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500


In [40]:
matched_rssd.drop('index',axis=1,inplace=True)

In [41]:
matched_rssd=pd.concat([matched_rssd,output[0]],axis=0)

In [42]:
unmatched_rssd=output[1]
within_fuzzys_dict=output[2]

In [43]:
output=fuzzy_match(unmatched_rssd,nic,'clean_name','clean_long_name',nmfp_nic_dict)

In [44]:
if 'level_0' in output[0].columns:
    output[0].drop('level_0',axis=1,inplace=True)

In [45]:
matched_rssd=pd.concat([matched_rssd,output[0]],axis=0)

In [46]:
unmatched_rssd=output[1]

In [47]:
matched_rssd.shape[0]

13886

In [48]:
matched_rssd.reset_index(inplace=True)

In [49]:
#pick the best fuzzy match when there are multiple
matched_rssd['best_fuzzy']=pd.Series(process.extractOne(matched_rssd['clean_name'][i],matched_rssd['matched_name'][i])[0] for i in matched_rssd.index)

In [51]:
matched_rssd.drop(['matched_name2'],axis=1,inplace=True)

In [74]:
process.extractOne(matched_rssd['clean_name'][i],matched_rssd['matched_name'][i])[0]

{'wells fargo bank na': 'wells fargo co',
 'barnegat twp brd of edu': 'barnegat',
 'princeton nj brd edu': 'princeton',
 'barclays bank plcgrand cayman': 'barclays bank plc',
 'bartlett tenn': 'bartlett',
 'capital one bank': 'capital one bank usa national association',
 'illinois state': 'state of illinois',
 'us treasury note': 'united states treasury department',
 'us treasury bill': 'united states treasury department',
 'united states treasury bond': 'united states treasury department',
 'federal home loan banks': 'federal home loan bank system',
 'stearns bank': 'stearns bank national association',
 'celtic bank': 'celtic bank corporation',
 'bmo harris bank': 'bmo harris bank national association',
 'rbc municipal products inc trust': 'rbc capital markets llc',
 'nuveen municipal market opportunity fund': 'nuveen municipal market opportunity fund inc',
 'university of virginia': 'university of virginia investment management company',
 'bmo harris bank na': 'bmo harris bank nation

In [50]:
#matched_name is either manual fix or best fuzzy
matched_rssd['matched_name']=pd.Series(clean_name_fixes_dict[matched_rssd['clean_name'][i]] if matched_rssd['clean_name'][i] in clean_name_fixes_dict.keys() else matched_rssd['best_fuzzy'][i] for i in matched_rssd.index)

In [51]:
matched_rssd.drop(['best_fuzzy'],inplace=True,axis=1)

In [52]:
#add in the manual matches from clean_name_fixes
temp=unmatched_rssd[unmatched_rssd['clean_name'].isin(clean_name_fixes_dict.keys())].copy()

In [53]:
temp.drop(['level_0','old_index'],axis=1,inplace=True)

In [54]:
temp['matched_name']=pd.Series()
temp['good_fuzzys']=pd.Series()

In [55]:
matched_rssd=pd.concat([matched_rssd,temp],axis=0)

In [56]:
#remove manual matches from unmatched
unmatched_rssd.drop(unmatched_rssd[unmatched_rssd['clean_name'].isin(clean_name_fixes_dict.keys())].index,inplace=True)

In [64]:
clean_name_matches=[process.extractBests(i,matched_rssd['trimmed_name'].values,limit=1,score_cutoff=90) for i in tqdm(unmatched_rssd['trimmed_name'])]




  0%|          | 0/11862 [00:00<?, ?it/s]


  0%|          | 1/11862 [00:00<3:06:21,  1.06it/s]


  0%|          | 2/11862 [00:01<3:03:36,  1.08it/s]


  0%|          | 3/11862 [00:02<3:01:27,  1.09it/s]


  0%|          | 4/11862 [00:03<2:59:25,  1.10it/s]


  0%|          | 5/11862 [00:04<2:58:10,  1.11it/s]


  0%|          | 6/11862 [00:05<2:59:23,  1.10it/s]


  0%|          | 7/11862 [00:06<3:13:37,  1.02it/s]


  0%|          | 8/11862 [00:07<3:12:06,  1.03it/s]


  0%|          | 9/11862 [00:08<3:14:01,  1.02it/s]


  0%|          | 10/11862 [00:09<3:08:16,  1.05it/s]


  0%|          | 11/11862 [00:10<3:11:16,  1.03it/s]


  0%|          | 12/11862 [00:11<3:12:57,  1.02it/s]


  0%|          | 13/11862 [00:12<3:43:51,  1.13s/it]


  0%|          | 14/11862 [00:14<4:04:58,  1.24s/it]


  0%|          | 15/11862 [00:15<4:20:37,  1.32s/it]


  0%|          | 16/11862 [00:18<5:33:38,  1.69s/it]


  0%|          | 17/11862 [00:19<5:08:15,  1.56s/it]


  0%|          | 18/11862 [

  2%|▏         | 294/11862 [05:35<3:19:35,  1.04s/it]


  2%|▏         | 295/11862 [05:36<3:32:29,  1.10s/it]


  2%|▏         | 296/11862 [05:38<4:07:28,  1.28s/it]


  3%|▎         | 297/11862 [05:39<3:44:20,  1.16s/it]


  3%|▎         | 298/11862 [05:41<4:58:53,  1.55s/it]


  3%|▎         | 299/11862 [05:42<4:32:45,  1.42s/it]


  3%|▎         | 300/11862 [05:43<4:21:01,  1.35s/it]


  3%|▎         | 301/11862 [05:44<3:55:20,  1.22s/it]


  3%|▎         | 302/11862 [05:45<3:36:01,  1.12s/it]


  3%|▎         | 303/11862 [05:46<3:20:34,  1.04s/it]


  3%|▎         | 304/11862 [05:47<3:08:29,  1.02it/s]


  3%|▎         | 305/11862 [05:48<3:00:52,  1.06it/s]


  3%|▎         | 306/11862 [05:48<2:56:14,  1.09it/s]


  3%|▎         | 307/11862 [05:50<3:08:22,  1.02it/s]


  3%|▎         | 308/11862 [05:51<3:28:09,  1.08s/it]


  3%|▎         | 309/11862 [05:52<3:34:07,  1.11s/it]


  3%|▎         | 310/11862 [05:53<3:20:02,  1.04s/it]


  3%|▎         | 311/11862 [05:54<3:36:25,  1.12

  5%|▍         | 586/11862 [10:37<2:51:26,  1.10it/s]


  5%|▍         | 587/11862 [10:38<2:47:40,  1.12it/s]


  5%|▍         | 588/11862 [10:39<2:52:42,  1.09it/s]


  5%|▍         | 589/11862 [10:40<2:53:48,  1.08it/s]


  5%|▍         | 590/11862 [10:41<2:50:55,  1.10it/s]


  5%|▍         | 591/11862 [10:42<2:46:46,  1.13it/s]


  5%|▍         | 592/11862 [10:43<2:56:33,  1.06it/s]


  5%|▍         | 593/11862 [10:44<3:04:28,  1.02it/s]


  5%|▌         | 594/11862 [10:45<3:13:41,  1.03s/it]


  5%|▌         | 595/11862 [10:46<3:14:32,  1.04s/it]


  5%|▌         | 596/11862 [10:47<3:18:16,  1.06s/it]


  5%|▌         | 597/11862 [10:48<3:06:34,  1.01it/s]


  5%|▌         | 598/11862 [10:49<2:58:39,  1.05it/s]


  5%|▌         | 599/11862 [10:50<3:07:31,  1.00it/s]


  5%|▌         | 600/11862 [10:51<2:59:10,  1.05it/s]


  5%|▌         | 601/11862 [10:52<2:54:34,  1.08it/s]


  5%|▌         | 602/11862 [10:52<2:47:35,  1.12it/s]


  5%|▌         | 603/11862 [10:53<2:43:21,  1.15

  7%|▋         | 878/11862 [15:41<3:16:30,  1.07s/it]


  7%|▋         | 879/11862 [15:43<3:27:54,  1.14s/it]


  7%|▋         | 880/11862 [15:44<3:22:52,  1.11s/it]


  7%|▋         | 881/11862 [15:45<3:12:58,  1.05s/it]


  7%|▋         | 882/11862 [15:46<3:23:14,  1.11s/it]


  7%|▋         | 883/11862 [15:47<3:24:40,  1.12s/it]


  7%|▋         | 884/11862 [15:48<3:24:14,  1.12s/it]


  7%|▋         | 885/11862 [15:50<4:14:41,  1.39s/it]


  7%|▋         | 886/11862 [15:51<4:07:44,  1.35s/it]


  7%|▋         | 887/11862 [15:52<3:50:50,  1.26s/it]


  7%|▋         | 888/11862 [15:53<3:27:20,  1.13s/it]


  7%|▋         | 889/11862 [15:54<3:12:15,  1.05s/it]


  8%|▊         | 890/11862 [15:56<4:13:39,  1.39s/it]


  8%|▊         | 891/11862 [15:57<3:50:59,  1.26s/it]


  8%|▊         | 892/11862 [15:58<3:30:53,  1.15s/it]


  8%|▊         | 893/11862 [15:59<3:23:12,  1.11s/it]


  8%|▊         | 894/11862 [16:01<3:37:56,  1.19s/it]


  8%|▊         | 895/11862 [16:02<3:26:32,  1.13

 10%|▉         | 1166/11862 [20:53<2:51:34,  1.04it/s]


 10%|▉         | 1167/11862 [20:54<3:34:52,  1.21s/it]


 10%|▉         | 1168/11862 [20:56<3:34:33,  1.20s/it]


 10%|▉         | 1169/11862 [20:57<3:44:29,  1.26s/it]


 10%|▉         | 1170/11862 [20:58<3:23:19,  1.14s/it]


 10%|▉         | 1171/11862 [20:59<3:11:18,  1.07s/it]


 10%|▉         | 1172/11862 [21:00<2:57:59,  1.00it/s]


 10%|▉         | 1173/11862 [21:00<2:52:12,  1.03it/s]


 10%|▉         | 1174/11862 [21:02<3:02:17,  1.02s/it]


 10%|▉         | 1175/11862 [21:02<2:53:09,  1.03it/s]


 10%|▉         | 1176/11862 [21:04<3:01:28,  1.02s/it]


 10%|▉         | 1177/11862 [21:05<2:57:46,  1.00it/s]


 10%|▉         | 1178/11862 [21:06<2:57:36,  1.00it/s]


 10%|▉         | 1179/11862 [21:06<2:51:37,  1.04it/s]


 10%|▉         | 1180/11862 [21:07<2:49:41,  1.05it/s]


 10%|▉         | 1181/11862 [21:08<2:44:58,  1.08it/s]


 10%|▉         | 1182/11862 [21:09<3:00:24,  1.01s/it]


 10%|▉         | 1183/11862 [21

...

 90%|█████████ | 10724/11862 [2:57:59<18:01,  1.05it/s]


 90%|█████████ | 10725/11862 [2:58:00<17:34,  1.08it/s]


 90%|█████████ | 10726/11862 [2:58:01<17:40,  1.07it/s]


 90%|█████████ | 10727/11862 [2:58:02<18:13,  1.04it/s]


 90%|█████████ | 10728/11862 [2:58:03<17:42,  1.07it/s]


 90%|█████████ | 10729/11862 [2:58:04<17:13,  1.10it/s]


 90%|█████████ | 10730/11862 [2:58:04<16:36,  1.14it/s]


 90%|█████████ | 10731/11862 [2:58:05<17:26,  1.08it/s]


 90%|█████████ | 10732/11862 [2:58:06<17:25,  1.08it/s]


 90%|█████████ | 10733/11862 [2:58:07<17:45,  1.06it/s]


 90%|█████████ | 10734/11862 [2:58:08<17:11,  1.09it/s]


 90%|█████████ | 10735/11862 [2:58:09<16:58,  1.11it/s]


 91%|█████████ | 10736/11862 [2:58:10<16:40,  1.13it/s]


 91%|█████████ | 10737/11862 [2:58:11<16:33,  1.13it/s]


 91%|█████████ | 10738/11862 [2:58:12<16:29,  1.14it/s]


 91%|█████████ | 10739/11862 [2:58:13<16:58,  1.10it/s]


 91%|█████████ | 10740/11862 [2:58:13<16:41,  1.12it/s]


 91%|█████████

 93%|█████████▎| 11006/11862 [3:02:21<12:47,  1.11it/s]


 93%|█████████▎| 11007/11862 [3:02:22<12:34,  1.13it/s]


 93%|█████████▎| 11008/11862 [3:02:23<13:53,  1.02it/s]


 93%|█████████▎| 11009/11862 [3:02:24<13:38,  1.04it/s]


 93%|█████████▎| 11010/11862 [3:02:25<13:08,  1.08it/s]


 93%|█████████▎| 11011/11862 [3:02:25<12:56,  1.10it/s]


 93%|█████████▎| 11012/11862 [3:02:26<13:07,  1.08it/s]


 93%|█████████▎| 11013/11862 [3:02:28<13:51,  1.02it/s]


 93%|█████████▎| 11014/11862 [3:02:28<13:08,  1.08it/s]


 93%|█████████▎| 11015/11862 [3:02:29<12:57,  1.09it/s]


 93%|█████████▎| 11016/11862 [3:02:30<12:36,  1.12it/s]


 93%|█████████▎| 11017/11862 [3:02:32<15:27,  1.10s/it]


 93%|█████████▎| 11018/11862 [3:02:33<14:29,  1.03s/it]


 93%|█████████▎| 11019/11862 [3:02:33<13:44,  1.02it/s]


 93%|█████████▎| 11020/11862 [3:02:34<14:05,  1.00s/it]


 93%|█████████▎| 11021/11862 [3:02:35<14:01,  1.00s/it]


 93%|█████████▎| 11022/11862 [3:02:36<13:30,  1.04it/s]


 93%|█████████

 95%|█████████▌| 11288/11862 [3:06:43<07:55,  1.21it/s]


 95%|█████████▌| 11289/11862 [3:06:44<07:57,  1.20it/s]


 95%|█████████▌| 11290/11862 [3:06:45<08:03,  1.18it/s]


 95%|█████████▌| 11291/11862 [3:06:46<08:00,  1.19it/s]


 95%|█████████▌| 11292/11862 [3:06:47<07:59,  1.19it/s]


 95%|█████████▌| 11293/11862 [3:06:48<08:35,  1.10it/s]


 95%|█████████▌| 11294/11862 [3:06:49<08:22,  1.13it/s]


 95%|█████████▌| 11295/11862 [3:06:49<08:15,  1.14it/s]


 95%|█████████▌| 11296/11862 [3:06:50<08:18,  1.13it/s]


 95%|█████████▌| 11297/11862 [3:06:51<08:10,  1.15it/s]


 95%|█████████▌| 11298/11862 [3:06:52<08:01,  1.17it/s]


 95%|█████████▌| 11299/11862 [3:06:53<08:09,  1.15it/s]


 95%|█████████▌| 11300/11862 [3:06:54<08:07,  1.15it/s]


 95%|█████████▌| 11301/11862 [3:06:55<09:02,  1.03it/s]


 95%|█████████▌| 11302/11862 [3:06:56<08:59,  1.04it/s]


 95%|█████████▌| 11303/11862 [3:06:57<08:34,  1.09it/s]


 95%|█████████▌| 11304/11862 [3:06:58<08:24,  1.11it/s]


 95%|█████████

 98%|█████████▊| 11570/11862 [3:11:15<04:31,  1.07it/s]


 98%|█████████▊| 11571/11862 [3:11:16<04:46,  1.01it/s]


 98%|█████████▊| 11572/11862 [3:11:17<04:32,  1.07it/s]


 98%|█████████▊| 11573/11862 [3:11:18<04:25,  1.09it/s]


 98%|█████████▊| 11574/11862 [3:11:19<04:27,  1.08it/s]


 98%|█████████▊| 11575/11862 [3:11:20<04:19,  1.11it/s]


 98%|█████████▊| 11576/11862 [3:11:21<04:19,  1.10it/s]


 98%|█████████▊| 11577/11862 [3:11:21<04:16,  1.11it/s]


 98%|█████████▊| 11578/11862 [3:11:22<04:08,  1.14it/s]


 98%|█████████▊| 11579/11862 [3:11:23<04:07,  1.14it/s]


 98%|█████████▊| 11580/11862 [3:11:24<04:12,  1.12it/s]


 98%|█████████▊| 11581/11862 [3:11:25<04:16,  1.10it/s]


 98%|█████████▊| 11582/11862 [3:11:26<04:12,  1.11it/s]


 98%|█████████▊| 11583/11862 [3:11:27<04:15,  1.09it/s]


 98%|█████████▊| 11584/11862 [3:11:28<04:16,  1.09it/s]


 98%|█████████▊| 11585/11862 [3:11:29<04:11,  1.10it/s]


 98%|█████████▊| 11586/11862 [3:11:30<04:10,  1.10it/s]


 98%|█████████

100%|█████████▉| 11852/11862 [3:15:46<00:09,  1.02it/s]


100%|█████████▉| 11853/11862 [3:15:47<00:08,  1.01it/s]


100%|█████████▉| 11854/11862 [3:15:48<00:07,  1.04it/s]


100%|█████████▉| 11855/11862 [3:15:49<00:06,  1.06it/s]


100%|█████████▉| 11856/11862 [3:15:49<00:05,  1.08it/s]


100%|█████████▉| 11857/11862 [3:15:50<00:04,  1.04it/s]


100%|█████████▉| 11858/11862 [3:15:51<00:03,  1.07it/s]


100%|█████████▉| 11859/11862 [3:15:52<00:02,  1.05it/s]


100%|█████████▉| 11860/11862 [3:15:53<00:01,  1.07it/s]


100%|█████████▉| 11861/11862 [3:15:54<00:00,  1.09it/s]


100%|██████████| 11862/11862 [3:15:55<00:00,  1.12it/s]




In [71]:
clean_name_matches2=[i[0][0] if len(i)>0 else i for i in clean_name_matches]

In [81]:
matched_name_matches=[]
for i in clean_name_matches2:
    if len(i)>0:
        matched_name_matches.append(matched_rssd['matched_name'][matched_rssd['trimmed_name'].values==i].values[0])
    else:
        matched_name_matches.append(i)

In [92]:
unmatched_rssd.drop(['level_0'],axis=1,inplace=True)

In [93]:
unmatched_rssd.reset_index(inplace=True)

In [94]:
unmatched_rssd['matched_name']=pd.Series(matched_name_matches)

In [101]:
temp=unmatched_rssd[unmatched_rssd['matched_name'].apply(lambda x: type(x)==str)].copy()

In [104]:
temp.drop(['level_0','old_index'],axis=1,inplace=True)

In [106]:
temp['good_fuzzys']=pd.Series()

In [108]:
matched_rssd=pd.concat([matched_rssd,temp],axis=0)

In [110]:
unmatched_rssd=unmatched_rssd[unmatched_rssd['matched_name'].apply(lambda x: type(x)!=str)]

In [115]:
matched_rssd.to_csv('matched.csv')
unmatched_rssd.to_csv('unmatched.csv')

In [116]:
matched_rssd[matched_rssd['clean_name']=='us treasury notes']

,index,percent,cumulative_percent,cusip6,isin,lei,cik,rssd_id,ticker,nmfp_name,...,frb_trace_name,nic_trace_name,markit_trace_name,ccm_name,edgar_name,clean_name,trimmed_name,full_stripped_name,matched_name,good_fuzzys
7,17,0.3153,34.205296,912828,NaN,254900HROIFWPRGM1V77,NaN,NaN,USGB,U.S. Treasury Notes,...,NaN,NaN,NaN,NaN,NaN,us treasury notes,us treasury notes,,NaN,NaN


In [117]:
sum(matched_rssd['percent'])

92.60465185758945